In [ ]:
!pip install opencv-python
!pip install opencv-python-headless

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time
import matplotlib.pyplot as plt
import socket

# Cargar el modelo 
#La ruta es dependiendo donde se encuentre el archivo h5 del modelo entrenado
ruta_modelo = 'C:\\Users\\ferna\\OneDrive\\Documentos\\ElMejorModelo-2024-02-16.h5'
best_model = load_model(ruta_modelo)

def TakeSnapshot(filename='photo.jpg'):     # Función para capturar una foto
    cap = cv2.VideoCapture(0)               # Abrir la cámara
    ret, frame = cap.read()                 # Capturar un frame de la cámara
    cv2.imwrite(filename, frame)            # Guardar el frame como una imagen en el archivo 
    cap.release()                           # Liberar los recursos de la cámara
    return filename                         # Devolver el nombre del archivo en el que se guardó la imagen

def detectar_expresion(s):                  # Función para detectar expresiones faciales
    try:                                    # Toma una foto 
        filename = TakeSnapshot()           # Toma la foto y guarda en un archivo
        print('Guardada en {}'.format(filename))    # Muestra el nombre donde fue guardada la imagen
        image = cv2.imread(filename)        # Lee la imagen recién capturada
        caras = face_cascade.detectMultiScale(image, 1.1, 4)    # Detecta caras en la imagen
        if len(caras) > 0:                  # Si se detecta al menos una cara   
            for (x, y, w, h) in caras:      # Por cada cara detectada, realiza la predicción de expresión
                cara = image[y:y+h, x:x+w]
                cara = cv2.resize(cara, (28, 28))
                cara = cara / 255.0
                cara = np.expand_dims(cara, axis=0)
                
                # Predice la expresión facial
                prediccion = best_model.predict(cara)[0]
                print(prediccion)
                
                # Obtiene la emoción predicha
                indice_emocion_predicha = np.argmax(prediccion)
                emocion_predicha = emociones[indice_emocion_predicha]
                nombre_emocion_predicha = diccionario_emociones[emocion_predicha]['nombre']
                valor_emocion_predicha = diccionario_emociones[emocion_predicha]['valor']

                # Imprime la emoción detectada y muestra la imagen capturada con matplotlib
                print(f"Emoción inferida: {nombre_emocion_predicha}")                
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                plt.title(f"Emoción inferida: {nombre_emocion_predicha}")
                plt.axis('off')
                plt.show()

                # Envía el valor de la emoción predicha a través del socket
                comando = valor_emocion_predicha
                s.send(comando.encode())
                
    except Exception as err:
        print(str(err))

# Configuración del socket
s = socket.socket()
s.connect(("192.168.0.116", 2020)) #La direccion IP es dependiendo la que muestre el servidor, y el puerto es propuesto por mi

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# Emociones disponibles
emociones = ['Angry', 'Happy', 'Neutral', 'Sad']
# Diccionario de emociones
diccionario_emociones = {
    'Angry':    {'nombre': 'Enojo',     'valor': '1'},
    'Happy':    {'nombre': 'Felicidad', 'valor': '2'},
    'Neutral':  {'nombre': 'Neutral',   'valor': '3'},
    'Sad':      {'nombre': 'Tristeza',  'valor': '4'}
}

# Tiempo de espera entre cada captura (en segundos)
tiempo_espera = 10  # 10 segundos

# Ejecuta un bucle continuo para tomar fotos cada 10 segundos
while True:
    detectar_expresion(s)  # Realiza la detección de expresiones faciales
    time.sleep(tiempo_espera)  # Espera 10 segundos antes de la próxima captura
